In [1]:
!ls /workspace/results/syn_baseline/dataflow_partition0_81fmxkyy/

df_model.onnx  df_model_fifos.onnx  df_model_ooc.onnx  df_model_stitchedip.onnx


In [3]:
showInNetron("/workspace/results/syn_baseline/end2end_quartznet_fifos.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/syn_baseline/end2end_quartznet_fifos.onnx' at http://0.0.0.0:8081


In [1]:
from finn.util.visualization import showInNetron

showInNetron("/workspace/results/syn_baseline/dataflow_partition0_81fmxkyy/df_model_fifos.onnx")

Serving '/workspace/results/syn_baseline/dataflow_partition0_81fmxkyy/df_model_fifos.onnx' at http://0.0.0.0:8081


In [2]:
showInNetron("/workspace/results/temp/bnn_pynq.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/temp/bnn_pynq.onnx' at http://0.0.0.0:8081


In [2]:
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.fpgadataflow.annotate_cycles import AnnotateCycles
from finn.custom_op.registry import getCustomOp

model = ModelWrapper("/workspace/results/syn_baseline/dataflow_partition0_81fmxkyy/df_model_fifos.onnx")
model = model.transform(AnnotateCycles())

max_cyc = 0
for n in model.graph.node:
    cycles = getCustomOp(n).get_nodeattr("cycles_estimate")
    print("{}\t{}".format(n.name, cycles))
    if cycles>max_cyc:
        max_cyc=cycles
    
print("MAX: {}".format(max_cyc))

StreamingFCLayer_Batch_0	131072
StreamingDataWidthConverter_Batch_0	0
DuplicateStreams_Batch_0	32768
StreamingDataWidthConverter_Batch_1	0
FMPadding_Batch_0	40960
StreamingDataWidthConverter_Batch_2	0
StreamingFCLayer_Batch_1	524288
StreamingDataWidthConverter_Batch_3	0
ConvolutionInputGenerator1D_0	280576
Vector_Vector_Activate_Batch_0	270336
StreamingFCLayer_Batch_2	524288
StreamingDataWidthConverter_Batch_4	0
FMPadding_Batch_1	40960
StreamingDataWidthConverter_Batch_5	0
ConvolutionInputGenerator1D_1	280576
Vector_Vector_Activate_Batch_1	270336
StreamingFCLayer_Batch_3	524288
StreamingDataWidthConverter_Batch_6	0
FMPadding_Batch_2	40960
StreamingDataWidthConverter_Batch_7	0
ConvolutionInputGenerator1D_2	280576
Vector_Vector_Activate_Batch_2	270336
StreamingFCLayer_Batch_4	524288
StreamingDataWidthConverter_Batch_8	0
FMPadding_Batch_3	40960
StreamingDataWidthConverter_Batch_9	0
ConvolutionInputGenerator1D_3	280576
Vector_Vector_Activate_Batch_3	270336
StreamingFCLayer_Batch_5	524288
S

# Verify outputs

In [3]:
labels = [" ", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m",
         "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "'"]
    
def __ctc_decoder_predictions_tensor(tensor, labels):
    """
    Decodes a sequence of labels to words
    """
    blank_id = len(labels)
    hypotheses = []
    labels_map = dict([(i, labels[i]) for i in range(len(labels))])
    decoded_prediction = []
    previous = len(labels)  # id of a blank symbol
    prediction = tensor
    for p in prediction:
        if (p != previous or previous == blank_id) and p != blank_id:
            decoded_prediction.append(p)
        previous = p
    hypothesis = ''.join([labels_map[c] for c in decoded_prediction])
    hypotheses.append(hypothesis)
    return hypotheses

In [4]:
import numpy as np

golden = np.load("/workspace/results/syn_baseline/librispeech_data/output_sample_float_0.npy", allow_pickle=True)
out_export = np.load("/workspace/results/syn_baseline/librispeech_data/quartznet_export_output.npy", allow_pickle=True)
out_tidy = dict(np.load("/workspace/results/syn_baseline/librispeech_data/quartznet_tidy_and_change_shape_tensors.npy", allow_pickle=True).tolist())
out_streamline = dict(np.load("/workspace/results/syn_baseline/librispeech_data/quartznet_streamline.npy", allow_pickle=True).tolist())
out_cpp = dict(np.load("/workspace/results/syn_baseline/librispeech_data/quartznet_cppsim.npy", allow_pickle=True).tolist())

print("Golden: {}".format(__ctc_decoder_predictions_tensor(golden.flatten(), labels)))
print("Exported: {}".format(__ctc_decoder_predictions_tensor(out_export.flatten(), labels)))
print("Tidy: {}".format(__ctc_decoder_predictions_tensor(out_tidy["global_out"].flatten(), labels)))
print("Streamline: {}".format(__ctc_decoder_predictions_tensor(out_streamline["global_out"].flatten(), labels)))
print("CPPSim: {}".format(__ctc_decoder_predictions_tensor(out_cpp["global_out"].flatten(), labels)))

Golden: ['vanity and vexation ev spad']
Exported: ['vanity and vexation of spad']
Tidy: ['vanity and vexation of spad']
Streamline: ['vanity and vexation of spat']
CPPSim: ['vanity and vexation of spat']


In [83]:
print("Golden: {}".format(__ctc_decoder_predictions_tensor(golden.flatten(), labels)))
a = __ctc_decoder_predictions_tensor(golden.flatten(), labels)

print("Expected: vanity and vexation of spat\nObtained: {}".format(a[0]))

b = "abcd.npy"
c = b.replace(".npy", "_decoded.txt")
print("{}\n{}".format(b,c))

Golden: ['vanity and vexation ev spad']
Expected: vanity and vexation of spat
Obtained: vanity and vexation ev spad
abcd.npy
abcd_decoded.txt


## VVAU weights/thresholds


In [16]:
import numpy as np
from finn.core.modelwrapper import ModelWrapper


model = ModelWrapper("/workspace/results/syn_baseline/dataflow_partition0_81fmxkyy/df_model_fifos.onnx")

for n in model.graph.node:
    if n.name=="Vector_Vector_Activate_Batch_32":
        weights = model.get_initializer(n.input[1])
        thresh = model.get_initializer(n.input[2])
        
for n in model.graph.node:
    if n.op_type=="Vector_Vector_Activate_Batch" and n.name!="Vector_Vector_Activate_Batch_32":
        w = model.get_initializer(n.input[1])
        t = model.get_initializer(n.input[2])
        if np.array_equal(weights, w):
            print(n.name)
            print("True")
        if np.array_equal(thresh, t):
            print(n.name)
            print("True")

In [19]:
for i in range(199):
    print('"StreamingDataWidthConverter_Batch_'+str(i)+'" : ' + '{"slr" : -},')

"StreamingDataWidthConverter_Batch_0" : {"slr" : -},
"StreamingDataWidthConverter_Batch_1" : {"slr" : -},
"StreamingDataWidthConverter_Batch_2" : {"slr" : -},
"StreamingDataWidthConverter_Batch_3" : {"slr" : -},
"StreamingDataWidthConverter_Batch_4" : {"slr" : -},
"StreamingDataWidthConverter_Batch_5" : {"slr" : -},
"StreamingDataWidthConverter_Batch_6" : {"slr" : -},
"StreamingDataWidthConverter_Batch_7" : {"slr" : -},
"StreamingDataWidthConverter_Batch_8" : {"slr" : -},
"StreamingDataWidthConverter_Batch_9" : {"slr" : -},
"StreamingDataWidthConverter_Batch_10" : {"slr" : -},
"StreamingDataWidthConverter_Batch_11" : {"slr" : -},
"StreamingDataWidthConverter_Batch_12" : {"slr" : -},
"StreamingDataWidthConverter_Batch_13" : {"slr" : -},
"StreamingDataWidthConverter_Batch_14" : {"slr" : -},
"StreamingDataWidthConverter_Batch_15" : {"slr" : -},
"StreamingDataWidthConverter_Batch_16" : {"slr" : -},
"StreamingDataWidthConverter_Batch_17" : {"slr" : -},
"StreamingDataWidthConverter_Batch_18"

# Apply SLR partitioning & count SDF nodes

In [42]:
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.general import ApplyConfig
from finn.transformation.fpgadataflow.floorplan import Floorplan
from finn.transformation.fpgadataflow.create_dataflow_partition import CreateDataflowPartition
from finn.util.basic import get_by_name

model = ModelWrapper("/workspace/results/syn_baseline/dataflow_partition0_81fmxkyy/df_model_fifos.onnx")
model = model.transform(ApplyConfig("/workspace/results/syn_baseline/folding_config_slr.json"))

for n in model.graph.node:
    slr = get_by_name(n.attribute, "slr").i
    name = n.name
    print("{}\t{}".format(name, slr))
    
model = model.transform(Floorplan())
model = model.transform(CreateDataflowPartition())

print(len(model.graph.node))

model.save("/tmp/slr.onnx")

    

StreamingFCLayer_Batch_0	0
DuplicateStreams_Batch_0	0
StreamingDataWidthConverter_Batch_0	0
FMPadding_Batch_0	0
StreamingDataWidthConverter_Batch_1	0
StreamingFCLayer_Batch_1	0
StreamingDataWidthConverter_Batch_2	0
ConvolutionInputGenerator1D_0	0
Vector_Vector_Activate_Batch_0	0
StreamingFCLayer_Batch_2	0
StreamingDataWidthConverter_Batch_3	0
FMPadding_Batch_1	0
StreamingDataWidthConverter_Batch_4	0
ConvolutionInputGenerator1D_1	0
Vector_Vector_Activate_Batch_1	0
StreamingFCLayer_Batch_3	0
StreamingDataWidthConverter_Batch_5	0
FMPadding_Batch_2	1
StreamingDataWidthConverter_Batch_6	1
ConvolutionInputGenerator1D_2	1
Vector_Vector_Activate_Batch_2	1
StreamingFCLayer_Batch_4	1
StreamingDataWidthConverter_Batch_7	1
FMPadding_Batch_3	1
StreamingDataWidthConverter_Batch_8	1
ConvolutionInputGenerator1D_3	1
Vector_Vector_Activate_Batch_3	1
StreamingFCLayer_Batch_5	1
StreamingDataWidthConverter_Batch_9	1
FMPadding_Batch_4	1
StreamingDataWidthConverter_Batch_10	1
ConvolutionInputGenerator1D_4	1


DuplicateStreams_Batch_0	0
StreamingDataWidthConverter_Batch_0	0
FMPadding_Batch_0	0
StreamingDataWidthConverter_Batch_1	0
StreamingFCLayer_Batch_1	0
StreamingDataWidthConverter_Batch_2	0
ConvolutionInputGenerator1D_0	0
Vector_Vector_Activate_Batch_0	0
StreamingFCLayer_Batch_2	0
StreamingDataWidthConverter_Batch_3	0
FMPadding_Batch_1	0
StreamingDataWidthConverter_Batch_4	0
ConvolutionInputGenerator1D_1	0
Vector_Vector_Activate_Batch_1	0
StreamingFCLayer_Batch_3	0
StreamingDataWidthConverter_Batch_5	0
0-------
StreamingDataWidthConverter_Batch_6	1
ConvolutionInputGenerator1D_2	1
Vector_Vector_Activate_Batch_2	1
StreamingFCLayer_Batch_4	1
StreamingDataWidthConverter_Batch_7	1
FMPadding_Batch_3	1
StreamingDataWidthConverter_Batch_8	1
ConvolutionInputGenerator1D_3	1
Vector_Vector_Activate_Batch_3	1
StreamingFCLayer_Batch_5	1
StreamingDataWidthConverter_Batch_9	1
FMPadding_Batch_4	1
StreamingDataWidthConverter_Batch_10	1
ConvolutionInputGenerator1D_4	1
Vector_Vector_Activate_Batch_4	1
Strea

DuplicateStreams_Batch_8	2
StreamingDataWidthConverter_Batch_97	2
FMPadding_Batch_40	2
StreamingDataWidthConverter_Batch_98	2
StreamingFCLayer_Batch_49	2
StreamingDataWidthConverter_Batch_99	2
ConvolutionInputGenerator1D_40	2
Vector_Vector_Activate_Batch_40	2
StreamingFCLayer_Batch_50	2
StreamingDataWidthConverter_Batch_100	2
FMPadding_Batch_41	2
StreamingDataWidthConverter_Batch_101	2
ConvolutionInputGenerator1D_41	2
Vector_Vector_Activate_Batch_41	2
StreamingFCLayer_Batch_51	2
StreamingDataWidthConverter_Batch_102	2
FMPadding_Batch_42	2
StreamingDataWidthConverter_Batch_103	2
ConvolutionInputGenerator1D_42	2
Vector_Vector_Activate_Batch_42	2
StreamingFCLayer_Batch_52	2
StreamingDataWidthConverter_Batch_104	2
FMPadding_Batch_43	2
StreamingDataWidthConverter_Batch_105	2
ConvolutionInputGenerator1D_43	2
Vector_Vector_Activate_Batch_43	2
StreamingFCLayer_Batch_53	2
StreamingDataWidthConverter_Batch_106	2
FMPadding_Batch_44	2
StreamingDataWidthConverter_Batch_107	2
ConvolutionInputGenerat

StreamingDataWidthConverter_Batch_192	0
FMPadding_Batch_74	0
StreamingDataWidthConverter_Batch_193	0
ConvolutionInputGenerator1D_74	0
Vector_Vector_Activate_Batch_74	0
StreamingDataWidthConverter_Batch_194	0
20-------
StreamingDataWidthConverter_Batch_195	0
AddStreams_Batch_14	0
Thresholding_Batch_14	0
FMPadding_Batch_75	0
StreamingDataWidthConverter_Batch_196	0
ConvolutionInputGenerator1D_75	0
Vector_Vector_Activate_Batch_75	0
StreamingDataWidthConverter_Batch_197	0
21-------
StreamingDataWidthConverter_Batch_198	0
22-------
StreamingFCLayer_Batch_93	0
23-------
24


In [84]:
showInNetron("/tmp/slr.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/slr.onnx' at http://0.0.0.0:8081


In [86]:
from finn.transformation.general import GiveUniqueNodeNames

model = ModelWrapper("/tmp/slr.onnx")
model = model.transform(GiveUniqueNodeNames())

model.save("/tmp/slr_named.onnx")
showInNetron("/tmp/slr_named.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/slr_named.onnx' at http://0.0.0.0:8081


In [85]:
showInNetron("/tmp/finn_dev_mirza/dataflow_partition6_rgmwvhvs/df_model.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/finn_dev_mirza/dataflow_partition6_rgmwvhvs/df_model.onnx' at http://0.0.0.0:8081


In [52]:
from finn.core.modelwrapper import ModelWrapper
from finn.custom_op.registry import getCustomOp
from finn.transformation.infer_data_layouts import InferDataLayouts
from finn.transformation.fpgadataflow.insert_iodma import InsertIODMA
from finn.transformation.fpgadataflow.insert_dwc import InsertDWC
from finn.transformation.general import GiveUniqueNodeNames
from finn.transformation.general import GiveReadableTensorNames
from finn.transformation.fpgadataflow.floorplan import Floorplan
from finn.transformation.fpgadataflow.create_dataflow_partition import CreateDataflowPartition

model = ModelWrapper("/workspace/results/syn_baseline/dataflow_partition0_81fmxkyy/df_model_fifos.onnx")

in_tensor = model.graph.node[0].input[0]
model.set_tensor_layout(in_tensor, ["N", "H", "W", "C"])
model = model.transform(InferDataLayouts())
# prepare at global level, then break up into kernels
prep_transforms = [InsertIODMA(512), InsertDWC()]
for trn in prep_transforms:
    model = model.transform(trn)
    model = model.transform(GiveUniqueNodeNames())
    model = model.transform(GiveReadableTensorNames())

print("Running Floorplan...")
model = model.transform(Floorplan(floorplan="/workspace/results/syn_baseline/folding_config_slr_copy.json"))

print("Running CreateDataflowPartition...")
model = model.transform(CreateDataflowPartition())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
# Save model
model.save("/tmp/df_model_vitisbuild.onnx")

Running Floorplan...


/workspace/finn-base/src/finn/transformation/general.py:331: UserWarning: 
No HW configuration for nodes: IODMA_0, IODMA_1
  "\nNo HW configuration for nodes: " + ", ".join(missing_configurations)
/workspace/finn/src/finn/transformation/fpgadataflow/floorplan.py:124: UserWarning: 2 nodes have no entry in the provided floorplan and no default value was set
  + "and no default value was set"


Running CreateDataflowPartition...
0-------
1-------
DuplicateStreams_Batch_0	0
StreamingDataWidthConverter_Batch_0	0
FMPadding_Batch_0	0
StreamingDataWidthConverter_Batch_1	0
StreamingFCLayer_Batch_1	0
StreamingDataWidthConverter_Batch_2	0
ConvolutionInputGenerator1D_0	0
Vector_Vector_Activate_Batch_0	0
StreamingFCLayer_Batch_2	0
StreamingDataWidthConverter_Batch_3	0
FMPadding_Batch_1	0
StreamingDataWidthConverter_Batch_4	0
ConvolutionInputGenerator1D_1	0
Vector_Vector_Activate_Batch_1	0
StreamingFCLayer_Batch_3	0
StreamingDataWidthConverter_Batch_5	0
2-------
StreamingDataWidthConverter_Batch_6	1
ConvolutionInputGenerator1D_2	1
Vector_Vector_Activate_Batch_2	1
StreamingFCLayer_Batch_4	1
StreamingDataWidthConverter_Batch_7	1
FMPadding_Batch_3	1
StreamingDataWidthConverter_Batch_8	1
ConvolutionInputGenerator1D_3	1
Vector_Vector_Activate_Batch_3	1
StreamingFCLayer_Batch_5	1
StreamingDataWidthConverter_Batch_9	1
FMPadding_Batch_4	1
StreamingDataWidthConverter_Batch_10	1
ConvolutionInputG

DuplicateStreams_Batch_8	2
StreamingDataWidthConverter_Batch_97	2
FMPadding_Batch_40	2
StreamingDataWidthConverter_Batch_98	2
StreamingFCLayer_Batch_49	2
StreamingDataWidthConverter_Batch_99	2
ConvolutionInputGenerator1D_40	2
Vector_Vector_Activate_Batch_40	2
StreamingFCLayer_Batch_50	2
StreamingDataWidthConverter_Batch_100	2
FMPadding_Batch_41	2
StreamingDataWidthConverter_Batch_101	2
ConvolutionInputGenerator1D_41	2
Vector_Vector_Activate_Batch_41	2
StreamingFCLayer_Batch_51	2
StreamingDataWidthConverter_Batch_102	2
FMPadding_Batch_42	2
StreamingDataWidthConverter_Batch_103	2
ConvolutionInputGenerator1D_42	2
Vector_Vector_Activate_Batch_42	2
StreamingFCLayer_Batch_52	2
StreamingDataWidthConverter_Batch_104	2
FMPadding_Batch_43	2
StreamingDataWidthConverter_Batch_105	2
ConvolutionInputGenerator1D_43	2
Vector_Vector_Activate_Batch_43	2
StreamingFCLayer_Batch_53	2
StreamingDataWidthConverter_Batch_106	2
FMPadding_Batch_44	2
StreamingDataWidthConverter_Batch_107	2
ConvolutionInputGenerat

StreamingDataWidthConverter_Batch_192	0
FMPadding_Batch_74	0
StreamingDataWidthConverter_Batch_193	0
ConvolutionInputGenerator1D_74	0
Vector_Vector_Activate_Batch_74	0
StreamingDataWidthConverter_Batch_194	0
22-------
StreamingDataWidthConverter_Batch_195	0
AddStreams_Batch_14	0
Thresholding_Batch_14	0
FMPadding_Batch_75	0
StreamingDataWidthConverter_Batch_196	0
ConvolutionInputGenerator1D_75	0
Vector_Vector_Activate_Batch_75	0
StreamingDataWidthConverter_Batch_197	0
23-------
StreamingDataWidthConverter_Batch_198	0
24-------
StreamingFCLayer_Batch_93	0
25-------


In [58]:
showInNetron("/tmp/df_model_vitisbuild.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/df_model_vitisbuild.onnx' at http://0.0.0.0:8081


In [59]:
model = ModelWrapper("/tmp/df_model_vitisbuild.onnx")
print(model.model.metadata_props)

[key: "res_total_hls"
value: "{\'BRAM_18K\': 3057.0, \'FF\': 440891.0, \'LUT\': 847000.0, \'DSP48E\': 3208.0, \'URAM\': 213.0}"
, key: "floorplan_json"
value: "/workspace/results/syn_baseline/folding_config_slr_copy.json"
]


In [73]:
out_streamline = dict(np.load("/workspace/results/syn_baseline/librispeech_data/quartznet_streamline.npy", allow_pickle=True).tolist())
a = out_streamline["MultiThreshold_0_out0"]
a_tp = np.transpose(a, (0,2,3,1))

print(a.shape)
print(a_tp.shape)

np.save("/workspace/results/input0_test.npy", a_tp)

(1, 64, 128, 1)
(1, 128, 1, 64)


In [64]:
showInNetron("/workspace/results/syn_baseline/end2end_quartznet_streamline.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/syn_baseline/end2end_quartznet_streamline.onnx' at http://0.0.0.0:8081
